In [ ]:
import cv2
import numpy as np
import os
import random
from sklearn.ensemble import RandomForestClassifier
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# Đường dẫn dataset
PLATE_DATASET_PATH = "/content/drive/MyDrive/license_plate_recognition/plate_detection_dataset"
CHAR_DATASET_PATH = "/content/drive/MyDrive/license_plate_recognition/char_dataset"
TEST_IMAGE_PATH = "/content/drive/MyDrive/license_plate_recognition/test_image.jpg"

In [ ]:
# ================================================
# PREPROCESSING
# ================================================

def preprocess_plate(plate_img, target_size=(100, 50)):
    """Tiền xử lý ảnh biển số"""
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, target_size)
    equalized = cv2.equalizeHist(resized)
    return equalized

def extract_plate_features(img):
    """Trích xuất HOG cho biển số"""
    return hog(img, orientations=9, pixels_per_cell=(8, 8),
              cells_per_block=(2, 2), transform_sqrt=True)

def preprocess_char(char_img):
    """Tiền xử lý ký tự"""
    resized = cv2.resize(char_img, (36, 36))
    blur = cv2.GaussianBlur(resized, (3, 3), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh

def extract_char_features(char_img):
    """Trích xuất HOG + LBP cho ký tự"""
    hog_feat = hog(char_img, orientations=12, pixels_per_cell=(6, 6),
                  cells_per_block=(2, 2), transform_sqrt=True)

    lbp = local_binary_pattern(char_img, P=16, R=2, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 18), range=(0, 17))
    lbp_feat = hist.astype(np.float32) / hist.sum()

    return np.hstack([hog_feat, lbp_feat])

In [ ]:
# ================================================
# LOAD DATASET
# ================================================

def load_dataset_from_drive(dataset_path, is_plate=False):
    """Load dataset từ Google Drive với data augmentation"""
    features = []
    labels = []

    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue

        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)

            img = cv2.imread(img_path) if is_plate else cv2.imread(img_path, 0)
            if img is None:
                continue

            # Tiền xử lý
            processed = preprocess_plate(img) if is_plate else preprocess_char(img)

            # Data augmentation
            augmentations = [
                processed,
                cv2.flip(processed, 1)
            ]

            # Rotation augmentation with correct target size and center
            if is_plate:
                # For plates, processed image shape is (height, width)
                h, w = processed.shape
                center = (w // 2, h // 2)
                rot_matrix = cv2.getRotationMatrix2D(center, random.uniform(-15, 15), 1)
                rotated_img = cv2.warpAffine(processed, rot_matrix, (w, h))
                augmentations.append(rotated_img)
            else:
                # For characters, processed image shape is (36, 36)
                h, w = processed.shape
                center = (w // 2, h // 2)
                rot_matrix = cv2.getRotationMatrix2D(center, random.uniform(-15, 15), 1)
                rotated_img = cv2.warpAffine(processed, rot_matrix, (w, h))
                augmentations.append(rotated_img)

            # Gaussian noise augmentation
            augmentations.append(cv2.add(processed, np.random.normal(0, 25, processed.shape).astype(np.uint8)))

            for aug in augmentations:
                features.append(extract_plate_features(aug) if is_plate else extract_char_features(aug))
                labels.append(class_name)

    return np.array(features), np.array(labels)

In [ ]:
# ================================================
# HUẤN LUYỆN MODEL
# ================================================

def train_models():
    # Model phát hiện biển số (n_estimators reverted to 150 for better confidence based on analysis)
    print("Đang huấn luyện model phát hiện biển số...")
    X_plate, y_plate = load_dataset_from_drive(PLATE_DATASET_PATH, is_plate=True)
    plate_model = RandomForestClassifier(n_estimators=150, class_weight='balanced', n_jobs=-1)
    plate_model.fit(X_plate, y_plate)

    # Model nhận diện ký tự
    print("\nĐang huấn luyện model nhận diện ký tự...")
    X_char, y_char = load_dataset_from_drive(CHAR_DATASET_PATH)
    le = LabelEncoder()
    y_encoded = le.fit_transform(y_char)
    char_model = RandomForestClassifier(n_estimators=200, class_weight='balanced', n_jobs=-1)
    char_model.fit(X_char, y_encoded)

    return plate_model, char_model, le

In [ ]:
# ================================================
# NHẬN DIỆN
# ================================================

def detect_license_plate(img_path, plate_model):
    """Phát hiện biển số từ ảnh đầu vào"""
    img = cv2.imread(img_path)
    if img is None:
        print(f"Không thể đọc ảnh từ đường dẫn: {img_path}")
        return None

    # Tiền xử lý
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 50, 200)

    # Tìm contours
    contours, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    best_plate = None
    max_prob = 0

    print("\n--- Đang xác định vùng chứa biển số ---")
    for i, contour in enumerate(contours):
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)

        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            candidate = img[y:y+h, x:x+w]

            # Ensure candidate is not empty
            if candidate.size == 0:
                continue

            # Dự đoán
            try:
                processed = preprocess_plate(candidate)
                features = extract_plate_features(processed)
                prob = plate_model.predict_proba([features])[0][1]
                print(f"Vùng chứa biển số {i+1} - Xác suất: {prob:.4f}")

                # Hiển thị biển số
                plt.figure(figsize=(4, 2))
                plt.imshow(cv2.cvtColor(candidate, cv2.COLOR_BGR2RGB))
                plt.title(f"Vùng chứa biển số {i+1} (Xác suất: {prob:.4f})")
                plt.axis('off')
                plt.show()

                if prob > max_prob:
                    max_prob = prob
                    best_plate = candidate
            except Exception as e:
                print(f"Lỗi xử lý ảnh {i+1}: {e}")
                continue

    print(f"--- Hoàn thành xác định biển số. Xác suất tốt nhất: {max_prob:.4f} ---")
    return best_plate if max_prob > 0.25 else None # Adjusted threshold to 0.25

def recognize_characters(plate_img, char_model, le):
    """Nhận diện các ký tự từ biển số"""
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY_INV, 11, 4)

    # Tìm contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    chars = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if 0.2 < w/h < 0.9 and 100 < cv2.contourArea(contour) < 2000:
            char_img = thresh[y:y+h, x:x+w]
            char_img = cv2.copyMakeBorder(char_img, 5, 5, 5, 5, cv2.BORDER_CONSTANT, value=0)
            char_img = cv2.resize(char_img, (36, 36))
            chars.append((x, char_img))

    # Sắp xếp và nhận diện
    chars = sorted(chars, key=lambda c: c[0])
    result = []
    for x, char_img in chars:
        features = extract_char_features(char_img)
        proba = char_model.predict_proba([features])[0]
        if np.max(proba) > 0.7:
            pred = char_model.predict([features])[0]
            result.append(le.inverse_transform([pred])[0])

    return ''.join(result)

In [ ]:
# ================================================
# MAIN
# ================================================

def main():
    # Huấn luyện model
    plate_model, char_model, char_le = train_models()

    # Nhận diện ảnh test
    test_img = cv2.imread(TEST_IMAGE_PATH)
    if test_img is None:
        print("Không tìm thấy ảnh")
        return

    # Phát hiện biển số
    plate = detect_license_plate(TEST_IMAGE_PATH, plate_model)
    if plate is None:
        print("Không phát hiện được biển số")
        return

    # Nhận diện ký tự
    result = recognize_characters(plate, char_model, char_le)

    # Hiển thị kết quả
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
    plt.title("Ảnh gốc")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(plate, cv2.COLOR_BGR2RGB))
    plt.title(f"Biển số nhận diện: {result}")
    plt.axis('off')

    plt.show()

if __name__ == "__main__":
    main()